# Can I build a neural network to solve Mastermind, and how will it compare with Knuth's algorithm?

<div align="center">
    <img src="https://m.media-amazon.com/images/I/612QbEzSafL._AC_SL1500_.jpg"
         alt="Box top from Mastermind game."
         width="400px"
        />
</div>

## 2025-05-19: Intro

I started using Gemini about a month ago, so although I've never tried to build a neural network, recently I've been thinking more about them. While staring at the <a href="https://en.wikipedia.org/wiki/Mastermind_(board_game)">Mastermind</a> board my kids left out, I started thinking about whether I could build a neural network to solve it. This would be overpowered, of course. While researching the game, I quickly stumbled across Donald Knuth's algorithm, which I immediately stopped reading about so I could think it through and implement it myself.

The plan here, I think, is:
1. Build a game environment.
2. Implement Knuth's algorithm.
3. Build a neural network.

The game environment should be able to return the latest result or all results; I gather I shouldn't ask my neural network to manage the game's history. I may try to build the neural network myself using NumPy. I just discovered <a href="https://www.youtube.com/playlist?list=PLQVvvaa0QuDcjD5BAw2DxE6OF2tius3V3">this playlist</a>, and I've been meaning to revisit the first few videos of <a href="https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi">3blue1brown's</a> for the ?th time.

Initial thoughts for the game environment:

```python
import random

class Mastermind:
    def __init__(self, return_history=False, seed=None):
        self.secret = None
        self.return_history = return_history
        self.history = []
        self.reset(seed)

    def reset(self, seed=None):
        if seed is not None:
            random.seed(seed)
        self.secret = random.choices(range(1, 7), k=4)
        self.history.clear()

    def step(self, guess):
        # score guess; determine how many black and white pegs
        # append (guess, score) to history
        return self.history[:] if self.return_history else self.history[-1]
```

Now that I think about it, I'm not sure what data structure I'll need to feed into the neural network. But I can work that out when I get there.

## 2025-05-20: Game Environment

In [1]:
from collections import Counter
import random

In [2]:
class Mastermind:
    def __init__(self, return_history=False, seed=None):
        self.secret = None
        self.secret_counts = Counter()
        self.return_history = return_history
        self.history = []
        self.reset(seed)

    def reset(self, seed=None):
        self.secret_counts.clear()
        self.history.clear()
        if seed is not None:
            random.seed(seed)
        self.secret = random.choices(range(1, 7), k=4)
        self.secret_counts.update(self.secret)

    def step(self, guess):
        black = sum(self.secret[i] == guess[i] for i in range(4))
        white = sum((self.secret_counts & Counter(guess)).values()) - black
        score = (black, white)
        self.history.append((guess, score))
        return self.history[:] if self.return_history else score

One thing this class doesn't address so far is how to help carry out Knuth's algorithm. For each remaining possible guess $A$, I think I'll need to take each other remaining possible guess $B$ and say, if $B$ were the secret, what score would $A$ receive? And then group the $B$s by that score, and aggregate them to measure $A$'s suitability as the next guess.

I'm starting to think that I don't need an actual game environment to carry out Knuth's algorithm.

In any case, here's a test run that I solved manually:

In [3]:
M = Mastermind(seed='first test')

In [4]:
M.step((1, 2, 3, 4))

(0, 2)

In [5]:
M.step((2, 4, 5, 6))

(0, 3)

In [6]:
M.step((4, 5, 2, 2))

(1, 1)

In [7]:
M.step((6, 6, 4, 2))

(1, 1)

In [8]:
M.step((6, 1, 2, 5))

(0, 3)

In [9]:
M.step((5, 3, 6, 2))

(4, 0)

In [10]:
len(M.history)

6

During my neural network studying, I wondered how many output neurons I should have for Mastermind. I think I should use one-hot encoding, with each possible color of each peg having its own neuron, so I'll have $4 \times 6 = 24$ output neurons.

I'm starting to see a lot of possible issues with the input neurons, though. I could represent each previous guess and score with $4 \times 6 + 2 \times 5 = 34$ neurons, and there are a maximum of 9 previous guesses so that's $9 \times 34 = 306$ input neurons. Maybe that's a lot? But here are some definite concerns:
1. Much of the history will be empty/missing. I gather these inputs could be set to zero. Maybe the first layer biases as well?
2. The order of previous guesses doesn't matter. I feel that brute-forcing through this would make training exponentially more difficult/expensive. I asked Gemini for some topics to look into: permutation invariance, set processing, aggregation functions and pooling, embedding functions, and deep sets. Among others. I may have picked a project that doesn't lend itself to a simple neural network.
3. The different colors are interchangeable. This one, it seems, a neural network may actually be well suited to learn.

Also, yes, I know, the `step` method above returns multiple types.

## 2025-06-03: Knuth's algorithm, first attempt

### Trust

It took me until yesterday to really understand what Knuth's algorithm does and does not try to do. I think I had subconsciously been stuck on the following possibility:

Consider a game with 21 possibilities, two of which are $A$ and $B$. What if the possible results of $A$ split the remainder into groups of 16, 1, 1, 1, 1; and the possible results of $B$ split the remainder into groups of 4, 4, 4, 4, 4. The algorithm tells us to choose $B$. Now, what if these 21 possibilities were actually a subgame?

What if, in the previous step, $X$ led to remainders of our 21 as well as 10, 9, 8; while $Y$ led to remainders of 12, 12, 12, 12? The algorithm tells us to choose $Y$ and forget about $X$. Even if each one of the possibilities $Z$ after $Y$ turns out to be 8, 1, 1, 1; even though the 4 from $(X \rightarrow B)$ would be preferable to the 8 from $(Y \rightarrow Z)$, we've committed to $Y$.

Knuth's algorithm doesn't concern itself with this. It implicitly trusts that Mastermind is balanced in some way.

### Grouping (incorrectly, it turns out)

At each step Knuth's algorithm looks only one step ahead and picks a best arrangement. Without loss of generality, there are only five distinct partitions of the number 4, so we only need consider five first guesses. For subsequent guesses we'll achieve the same type of simplification by processing possibilities in sort order and only replacing the best one seen if we find one better.

I missed something at first, though. For comparison, when I play Wordle, if I'm trying to avoid losing, I typically use 15 different letters in my first three guesses, even if I've gotten some hits in earlier guesses. Knuth's algorithm sometimes takes this approach, but when reading about Knuth's algorithm, as soon as I started to get the shape of it, I stopped reading. I wanted to work out the implementation details for myself, but it hadn't sunk in that I needed to consider all possible subsequent guesses, even if they were no longer possible solutions.

I realized this after I had implemented an algorithm, run it, fixed a couple issues, and computed what seemed to be a full solution within just a few seconds. Then I played some sample games, and every so often my algorithm solved it in six guesses. After digging through the code a bit, without sharing my code I asked Gemini what the problem might be, and it suggested I may have missed this part of the algorithm, that I may need to consider guesses that have been ruled out as a potential solution.

I then corrected for this, but when I ran what I thought was now Knuth's real algorithm, it ran for a day without finishing. I had also struggled along the way to work out how I should store the tree I was computing, which made my code rather ugly. So I'm scrapping pretty much everything until I have a more efficient way to compute everything. Here's what I may want to keep:

In [11]:
from itertools import product

In [12]:
starting_possibilities = list(product(range(1, 7), repeat=4))

starting_guesses = [
    (1, 1, 1, 1),
    (1, 1, 1, 2),
    (1, 1, 2, 2),
    (1, 1, 2, 3),
    (1, 2, 3, 4),
]

In [13]:
def get_score(guess, possible):
    black = sum(guess[i] == possible[i] for i in range(4))
    white = sum((Counter(guess) & Counter(possible)).values()) - black
    return (black, white)

## 2025-06-04: Ideas on efficiency

If I understand correctly, Knuth's algorithm requires doing the following many times: Given some subset of remaining possible solutions and a guess, I need to count how often across those possible solutions that guess would be given each possible score, and determine the maximum count.

I think the most productive thing I could do is to be able to return a score extremely efficiently. Originally I thought I would cache the score function, which is useful, but the way I wrote the function feels like there would be a lot of Python overhead in retrieving from the cache. Two tuples where the individual values are numbers between 1 and 6 — should I go with bit packing?

Eventually I realized that once we've enumerated the states and know how they compare with each other, it doesn't really matter how we identify them, as long as we can translate back for display. There are only $1{,}296 \times 1{,}296 = 1{,}679{,}616$ different inputs to the function. These scores can be computed, stored in an array or list, and accessed by index. As I need to pull many of these values at once, I should use NumPy for its advanced indexing.

The scores in this array can be bitpacked. Again, it doesn't matter what the values in the scores array are, as long as we can identify the win condition and translate back.

In [41]:
def encode_guess(guess):
    index = 0
    for element in guess:
        index *= 6
        index += element - 1
    return index

def decode_guess(index):
    guess = []
    while len(guess) < 4:
        index, r = divmod(index, 6)
        guess.insert(0, r + 1)
    return tuple(guess)

assert all(guess == decode_guess(encode_guess(guess)) for guess in starting_possibilities)

In [42]:
def encode_score(score):
    black, white = score
    return 16 * black + white

def decode_score(score):
    return divmod(score, 16)

assert all((black, white) == decode_score(encode_score((black, white)))
           for black in range(5) for white in range(5))

WIN = encode_score((4, 0))

In [16]:
import numpy as np

In [45]:
%%time
scores = np.empty((6**4, 6**4), dtype=np.uint8)
for row, guess in enumerate(starting_possibilities):
    for col, solution in enumerate(starting_possibilities):
        if row > col:
            scores[row, col] = scores[col, row]
        else:
            scores[row, col] = encode_score(get_score(guess, solution))

CPU times: user 9.11 s, sys: 23 ms, total: 9.13 s
Wall time: 9.15 s


Hmm...that's pretty slow. I mean, I know I only need to compute the decision tree once, but I've always wanted to learn how to properly vectorize functions with NumPy. (Remember when I thought this was going to be about neural networks?)

## 2025-06-06: Efficiency, part 2

This is all Gemini. I haven't studied it long enough yet to fully understand it.

In [46]:
from numba import njit, prange

In [47]:
def vectorized_mastermind_score(guesses, secrets):
    num_guesses = guesses.shape[0]
    num_secrets = secrets.shape[0]
    num_pegs = guesses.shape[1]
    num_colors = 6

    guesses_expanded = guesses[:, np.newaxis, :]
    secrets_expanded = secrets[np.newaxis, :, :]

    exact_matches = (guesses_expanded == secrets_expanded)
    black_pegs_matrix = np.sum(exact_matches, axis=2, dtype=np.uint8)

    temp_guesses_for_white = np.broadcast_to(guesses_expanded, (num_guesses, num_secrets, num_pegs)).copy()
    temp_secrets_for_white = np.broadcast_to(secrets_expanded, (num_guesses, num_secrets, num_pegs)).copy()

    temp_guesses_for_white[exact_matches] = 0
    temp_secrets_for_white[exact_matches] = 0

    temp_guesses_flat = temp_guesses_for_white.reshape(-1, num_pegs)
    temp_secrets_flat = temp_secrets_for_white.reshape(-1, num_pegs)

    @njit
    def _count_matches_numba_single_pair(g_row, s_row, num_colors):
        g_counts = np.bincount(g_row[g_row > 0], minlength=num_colors + 1)[1:]
        s_counts = np.bincount(s_row[s_row > 0], minlength=num_colors + 1)[1:]
        return np.sum(np.minimum(g_counts, s_counts))

    @njit(parallel=True)
    def _calculate_white_pegs_numba_all_pairs(guesses_flat, secrets_flat, num_colors):
        num_pairs = guesses_flat.shape[0]
        white_pegs_flat_results = np.empty(num_pairs, dtype=np.uint8)
        for i in prange(num_pairs):
            white_pegs_flat_results[i] = _count_matches_numba_single_pair(
                guesses_flat[i], secrets_flat[i], num_colors
            )
        return white_pegs_flat_results

    white_pegs_flat_results = _calculate_white_pegs_numba_all_pairs(
        temp_guesses_flat, temp_secrets_flat, num_colors
    )

    white_pegs_matrix = white_pegs_flat_results.reshape(num_guesses, num_secrets)

    return black_pegs_matrix, white_pegs_matrix

def encode_mastermind_score_matrix(black_matrix, white_matrix):
    return black_matrix * 16 + white_matrix

In [49]:
%%time
starting = np.array(list(product(np.arange(1, 7), repeat=4)))
guesses = starting
secrets = starting
black_scores_matrix, white_scores_matrix = vectorized_mastermind_score(guesses, secrets)
final_scores_matrix = encode_mastermind_score_matrix(black_scores_matrix, white_scores_matrix)

CPU times: user 6.8 s, sys: 302 ms, total: 7.1 s
Wall time: 4.07 s


In [53]:
assert np.array_equal(final_scores_matrix, scores)

Okay, that is indeed faster. Next step will be rewriting the algorithm.

The options for the second guess can be limited similarly to for the first guess. I know the highest number appearing in the first guess will be 2, so for the second guess, we shouldn't use 4 unless we've also used 3, etc.

In [55]:
def check_wlog(guess):
    L = sorted(set(guess), reverse=True)
    if len(L) == 1:
        b = L[0]
    for i in range(len(L) - 1):
        a, b = L[i:i+2]
        if a > 3 and a - b > 1:
            return False
    if b > 3:
        return False
    return True

This should speed things up because it reduces the number of guesses to check to less than one-quarter of the total:

In [57]:
sum(check_wlog(guess) for guess in starting_possibilities), len(starting_possibilities)

(299, 1296)